In [17]:
import sys
import os
import mysql.connector
from mysql.connector import Error
import psycopg2
import pandas as pd
import gc

In [18]:
from sqlalchemy import create_engine

In [19]:
outpath='C:/Users/sai kiran Reddy/Desktop/my.csv'

In [20]:
table_list=['sai.my']

In [21]:
dbname_trgt = 'new'
host_trgt = 'localhost'
port_trgt = '5432'
user_trgt = 'postgres'
pwd_trgt = 'saikiran'

In [22]:
def connect_mysql():
    try:
        conn_src = mysql.connector.connect(host='localhost',
                                         database='sai',
                                         user='root',
                                         password='1234')

        print("You're connected to database")
        return conn_src
    
    except Error as e:
        print("Error while connecting to MySQL", e)

In [23]:
def extract_from_source(extr_qury,tab_name):
    print("read data into pandas dataframe")
    conn_src=connect_mysql()
    df=pd.read_sql_query(extr_qury,conn_src)
    df.to_csv(outpath,index=False)
#     df=pd.read_csv(outpath,index_col=None)
#     rec_cunt=df.count()
#     print("extracted "+rec_cunt+" from "+tab_name)
    return outpath,df


In [24]:
def pg_load_table(outpath, tab_name, dbname_trgt, host_trgt, port_trgt, user_trgt, pwd_trgt):
    '''
    This function upload csv to a target table
    '''
    try:
        conn = psycopg2.connect(dbname=dbname_trgt, host=host_trgt, port=port_trgt,\
         user=user_trgt, password=pwd_trgt)
        print("Connecting to Database")
        cur = conn.cursor()
        f = open(outpath, "r")
        # Truncate the table first
        cur.execute("Truncate {} Cascade;".format(tab_name))
        print("Truncated {}".format(tab_name))
        # Load table from the file with header
        cur.copy_expert("copy {} from STDIN CSV HEADER QUOTE '\"'".format(tab_name), f)
        cur.execute("commit;")
        print("Loaded data into {}".format(tab_name))
        conn.close()
        print("DB connection closed.")

    except Exception as e:
        print("Error: {}".format(str(e)))
        sys.exit(1)
    

In [25]:
if __name__ == '__main__':
    for tab_name in table_list:
        print("generating extract query for "+tab_name+" table")
        extr_qury= "select * from "+tab_name+" ;"
        print(extr_qury)
        df=extract_from_source(extr_qury,tab_name)
        print("loading data into target table")
        tab_name="second"
        pg_load_table(outpath, tab_name, dbname_trgt, host_trgt, port_trgt, user_trgt, pwd_trgt)
        del df
        gc.collect()
        os.remove(outpath)


generating extract query for sai.my table
select * from sai.my ;
read data into pandas dataframe
You're connected to database
loading data into target table
Connecting to Database
Truncated second
Loaded data into second
DB connection closed.


In [13]:
print(extr_qury)

select * from sai.my ;
